In [11]:
import constti
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

def inputFPL():
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()
        
        
    p1 = requests.get(url1)
    #page1 = BeautifulSoup(p1.text)
    #data1 = str(page1.p)[3:-4]
    data1 = p1.text
    
    d1 = json.loads(data1)
    bigTable = pd.DataFrame(d1['elements'])
    bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
                   'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
                   'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
                   'value_season', 'form', 'value_form', 'points_per_game', 
                   'goals_conceded', 
                   'in_dreamteam', 'dreamteam_count',
                   'now_cost', 'cost_change_event', 'cost_change_event_fall',
                   'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
                   'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
                   'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
                   'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
    bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
    bigTable.to_csv(Path('in/fpltable.csv'))

    p4 = requests.get(url4)
    #page4 = BeautifulSoup(p4.text)
    #data4 = str(page4.p)[3:-4]
    d4 = json.loads(p4.text)
    Fixtures = pd.DataFrame(d4)
    Fixtures.to_csv(Path('in/fplfixtures.csv'))
    
    firstr = len(Fixtures)+1
    lastr = 0
    for i in range(len(Fixtures)):
        if Fixtures.at[i,'finished']==True:
            firstr = min(firstr, i)
            lastr = i
    if firstr < len(Fixtures)+1:
        lastGW = int(Fixtures.at[lastr,'event'])
    else: lastGW = 0
    if lastr == len(Fixtures):
        lastGW = int(Fixtures.at[lastr,'event'])







    Gameweeks = pd.DataFrame()
    for i in range(1,2*team_number - 1):
        url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
        p = requests.get(url)
        #page = BeautifulSoup(p.text)
        #data = str(page.p)[3:-4]
        d = json.loads(p.text)
        nexTour = pd.DataFrame(d['elements'])

        if not nexTour.empty:
            nt1 = pd.DataFrame(nexTour['stats'].tolist())
            nt1['id'] = nexTour['id']
            nt1['gameweek'] = i
            nt1['fixture'] = [nexTour.at[i,'explain'][0]['fixture'] if not nexTour.at[i,'explain']==[] \
                              else '' for i in nexTour.index]
            #nt1.index = nt1['gameweek']*1000+nt1['id']
            Gameweeks = Gameweeks.append(nt1, ignore_index=True)
            print(i)


    teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
    players = dict(zip(bigTable['id'],bigTable['full_name']))
    teamplayers = dict(zip(bigTable['id'],bigTable['team']))

    Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

    Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
    Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

    Gameweeks['team_a'] = [Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'] \
                                for i in Gameweeks.index]


    #Kill postponed matches lines

    #li =[]
    #for i in Gameweeks.index:
    #    if Gameweeks.at[i,'team_a'].empty:
    #        li.append(i)
    #Gameweeks = Gameweeks.drop(li)

    li =[]
    for i in Gameweeks.index:
        if not is_finished(Gameweeks.at[i,'fixture']):
            li.append(i)
    Gameweeks = Gameweeks.drop(li)


    Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a']) for i in Gameweeks.index]

    Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h']) \
                                for i in Gameweeks.index]

    Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else Gameweeks.at[i,'team_h'] \
                               for i in Gameweeks.index]
    Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else 'away' \
                               for i in Gameweeks.index]

    del Gameweeks['team_a']
    del Gameweeks['team_h']


    Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
    return d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers

if __name__ == '__main__':
    d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL()
    display(Gameweeks)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,threat,ict_index,total_points,in_dreamteam,id,gameweek,fixture,team,teamAgainst,side
0,90,0,0,1,0,0,0,0,0,0,...,0.0,1.8,7,False,14,1,10,1,13,away
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,27,1,6,2,17,away
2,90,0,0,1,0,0,0,0,0,0,...,0.0,2.5,9,False,47,1,7,4,18,away
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,48,1,7,4,18,away
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,72,1,2,3,15,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12241,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,387,22,218,19,15,away
12248,90,0,0,0,1,0,0,0,0,0,...,14.0,2.2,2,False,457,22,218,19,15,away
12256,83,1,0,1,0,0,0,0,0,0,...,21.0,7.5,7,True,501,22,218,15,19,home
12261,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,514,22,218,19,15,away
